In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.TTF").get_name()
plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import urllib.parse
from bs4 import BeautifulSoup
import requests
import pprint
import json

from pandas.io.json import json_normalize

# 1. 서울 혼잡빈도강도(읍면동단위)

## 1) Data Prepared

- 서울 혼잡빈도강도 + 동단위 인구(join)
- 서울 혼잡시간강도 + 동단위 인구(join)

In [25]:
# 서울 법정동별 인구현황 데이터 불러오기

pop_seoul_dong = pd.read_csv("data/pop_seoul_dong.csv")
pop_seoul_dong.head()

,자치구,동,인구수,여자,남자
0,종로구,사직동,9630,4335,5295
1,종로구,삼청동,2741,1312,1429
2,종로구,부암동,9702,4654,5048
3,종로구,평창동,18272,8518,9754
4,종로구,무악동,8245,3823,4422


In [26]:
pop_seoul_dong=pop_seoul_dong.set_index(['자치구','동'])

In [27]:
pop_seoul_dong

인구수     여자     남자
자치구 동                        
종로구 사직동    9630   4335   5295
    삼청동    2741   1312   1429
    부암동    9702   4654   5048
    평창동   18272   8518   9754
    무악동    8245   3823   4422
...         ...    ...    ...
강동구 천호2동  33671  16467  17204
    길동    45863  22374  23489
    강일동   32947  15789  17158
    상일1동  38832  19137  19695
    상일2동  10254   4812   5442

[426 rows x 3 columns]

In [114]:
traffic_jam = pd.read_csv("data/2022 서울_혼잡빈도강도_읍면동.csv")
traffic_jam.head()

,행정구역코드,시도명,시군구명,읍면동명,평일주말구분,혼잡빈도강도
0,1101053,서울특별시,종로구,사직동,평일,90.43
1,1101054,서울특별시,종로구,삼청동,평일,32.45
2,1101055,서울특별시,종로구,부암동,평일,60.51
3,1101056,서울특별시,종로구,평창동,평일,45.55
4,1101057,서울특별시,종로구,무악동,평일,36.04


## 2) 혼잡빈도강도(동별)

In [115]:
traffic_jam = traffic_jam[traffic_jam["시도명"] == "서울특별시"]

In [116]:
traffic_jam.drop(["행정구역코드", "시도명", "평일주말구분","시군구명"], axis=1,inplace=True)

In [117]:
traffic_jam = traffic_jam.rename({"읍면동명":"동"}, axis="columns")
traffic_jam.head()

,동,혼잡빈도강도
0,사직동,90.43
1,삼청동,32.45
2,부암동,60.51
3,평창동,45.55
4,무악동,36.04


In [118]:
traffic_jam=traffic_jam.set_index("동")

In [119]:
traffic_jam

,혼잡빈도강도
동,
사직동,90.43
삼청동,32.45
부암동,60.51
평창동,45.55
무악동,36.04
...,...
둔촌1동,21.40
둔촌2동,28.98
암사1동,39.04


In [123]:
traffic_jam_dong = pd.merge(pop_seoul_dong,traffic_jam,how='inner', on="동")

In [124]:
traffic_jam_dong

,인구수,여자,남자,혼잡빈도강도
동,,,,
사직동,9630,4335,5295,90.43
삼청동,2741,1312,1429,32.45
부암동,9702,4654,5048,60.51
평창동,18272,8518,9754,45.55
무악동,8245,3823,4422,36.04
...,...,...,...,...
둔촌2동,24641,12123,12518,28.98
암사1동,34643,17157,17486,39.04
천호2동,33671,16467,17204,42.78


In [125]:
traffic_jam_dong.to_csv("data/traffic_jam_dong.csv", encoding="utf-8")

## 3) 혼잡시간강도 (동별)

In [128]:
traffic_time = pd.read_csv("data/2022 서울_혼잡시간강도_읍면동.csv")
traffic_time.head()

,행정구역코드,시도명,시군구명,읍면동명,평일주말구분,혼잡시간강도
0,1101053,서울특별시,종로구,사직동,평일,95.12
1,1101054,서울특별시,종로구,삼청동,평일,53.03
2,1101055,서울특별시,종로구,부암동,평일,74.40
3,1101056,서울특별시,종로구,평창동,평일,59.39
4,1101057,서울특별시,종로구,무악동,평일,67.78


In [129]:
traffic_time = traffic_time[traffic_time["시도명"] == "서울특별시"]

In [130]:
traffic_time.drop(["행정구역코드", "시도명", "평일주말구분","시군구명"], axis=1,inplace=True)

In [131]:
traffic_time = traffic_time.rename({"읍면동명":"동"}, axis="columns")
traffic_time.head()

,동,혼잡시간강도
0,사직동,95.12
1,삼청동,53.03
2,부암동,74.40
3,평창동,59.39
4,무악동,67.78


In [132]:
traffic_time=traffic_time.set_index("동")
traffic_time.head()

,혼잡시간강도
동,
사직동,95.12
삼청동,53.03
부암동,74.40
평창동,59.39
무악동,67.78


In [133]:
traffic_time_dong = pd.merge(pop_seoul_dong,traffic_time,how='inner', on="동")

In [134]:
traffic_time_dong

,인구수,여자,남자,혼잡시간강도
동,,,,
사직동,9630,4335,5295,95.12
삼청동,2741,1312,1429,53.03
부암동,9702,4654,5048,74.40
평창동,18272,8518,9754,59.39
무악동,8245,3823,4422,67.78
...,...,...,...,...
둔촌2동,24641,12123,12518,54.40
암사1동,34643,17157,17486,68.03
천호2동,33671,16467,17204,59.84


In [135]:
traffic_jam_dong.to_csv("data/traffic_time_dong.csv", encoding="utf-8")